# The importance of Data Preprocessing

Artificial intelligence, machine learning, deep learning... Termen die steeds aan populariteit winnen in de wereld van technologie en ver daarbuiten. Er zijn dan ook oneindig veel toepassingen mogelijk en al gekend in verschillende sectoren. Er is dan veel sprake over de modellen en technieken die gebruikt zijn om een zeker probleem aan te pakken met behulp van AI, maar waar helaas veel minder over gesproken wordt is al het werk dat vooraf gaat aan het gebruik van dat model of die techniek: het verwerken en _cleanen_ van de nodige data.

Haast iedere toepassing van artificiele intelligentie heeft een massa aan data nodig om accuraat te zijn en zijn werk tot een goed einde te brengen. Voor een demo van een zekere techniek vinden we vaak mooie datasets die meteen te gebruiken zijn, maar helaas is dat in de praktijk zelden het geval. In de ontwikkeling van AI toepassingen gaat er vaak meer tijd verloren aan het verwerken van data dan aan het ontwikkelen van een model (dat is buiten de uren voor het trainen van het model gerekend). Dat deel van een project noemen we _data preprocessing_.

In de praktijk is vaak te zien dat de data niet meteen geschikt is en er datapunten ontbreken of er heel wat fouten inzitten. Daarnaast rust AI op wiskundige formules en heel wat data kan niet zomaar in die formules gegoten worden. Denk aan woorden/zinnen, afbeeldingen, geluidsbestanden... Maar ook gewoon numerieke gegevens die niet op dezelfde schaal zitten en dus een foute verhouding weergeven. 

Met deze blog wil ik graag dat deel dat in ieder AI project terug te vinden is wat meer aandacht geven. Daarom heb ik een dataset gezocht waar nog heel wat werk aan is voor het in een model gebruikt kan worden. Doorheen de blog overloop ik de verschillende die hierbij komen kijken. Uiteraard is dit voor ieder project of iedere dataset anders. Er zijn ontzettend veel technieken gekend voor _data preprocessing_, elk goed voor specifieke toepassingen met bijhorende voor- en nadelen. In deze blog zullen dus niet alle mnogelijkheden gebruikt worden, maar wordt hopelijk wel duidelijk hoeveel er komt kijken bij dit proces.

## Import libraries and data

De eerste stap is altijd om de nodige _libraries_ te importeren. Voor deze notebook koos ik voor de _pandas library_ om met de data te werken, voor _matplotlib_ voor visualisaties en voor _numpy_ voor zijn gebruiksgemak, goede datastructuren en een makkelijk gebruik van NaN waardes.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Daarna is het tijd om de data in te laden. Omdat de data verspreid is over meerdere files, doe ik dit hier nog niet. Ik voorzie wel al de info om verderop snel en makkelijk de juiste bestanden te kunnen uitlezen. Daarom geef ik het pad naar de juiste map mee en een lijst van de continenten die elk gelinkt kunnen worden aan een bestand.

In [2]:
PATH = './agriculture-crop-production/'
continents = ['America', 'Africa', 'Asia', 'Europe', 'Oceania']

De data gaat over landbouw van 1961 tot en met 2014 over de verschillende continenten (zonder Alaska).
De huidige opzet van de data is zeer inefficient om snel gegevens op te vragen volgens bepaalde zoektermen. Ter illustratie laad ik alvast het kleinste bestand, dat van Oceanie, in in een _pandas DataFrame_.

In [20]:
df = pd.read_csv(PATH + 'Oceania_reformed.csv', engine='python')

# TODO: wat is engine?

In [4]:
df.head(10)

,Area Code,Area,Item Code,Item,Year,YearF,Area harvested,Yield,Production,Seed
0,5,American Samoa,486,Bananas,1961,F,500.0,12000.0,600.0,NaN
1,5,American Samoa,486,Bananas,1962,F,500.0,12000.0,600.0,NaN
2,5,American Samoa,486,Bananas,1963,F,500.0,14400.0,720.0,NaN
3,5,American Samoa,486,Bananas,1964,F,500.0,22600.0,1130.0,NaN
4,5,American Samoa,486,Bananas,1965,NaN,405.0,22395.0,907.0,NaN
5,5,American Samoa,486,Bananas,1966,NaN,486.0,13992.0,680.0,NaN
6,5,American Samoa,486,Bananas,1967,F,550.0,21436.0,1179.0,NaN
7,5,American Samoa,486,Bananas,1968,F,630.0,17857.0,1125.0,NaN
8,5,American Samoa,486,Bananas,1969,F,700.0,27286.0,1910.0,NaN
9,5,American Samoa,486,Bananas,1970,F,700.0,33829.0,2368.0,NaN


### De kolommen hervormen

Dit eerste deel is geen standaard onderdeel van _data preprocessing_, maar gezien de inefficiënte indeling van de dataset wil ik starten met deze opnieuw in te delen

Momenteel zijn er voor ieder jaar twee kolommen, een voor de gegevens en een die aangeeft of de gegevens exact juist zijn of op een andere manier geschat zijn. Daarnaast is er een kolom die aangeeft of een zekere rij data weergeeft van '_production_', '_area harvested_', '_yield_' of '_seed_'. Dit resulteert in een totaal van maar liefst 115 kolommen.

Daarom lijkt het me beter en efficiënter om 1 kolom te maken voor 'jaar' en aparte kolommen voor '_production_', '_area harvested_', '_yield_' en '_seed_'. Dit is echter ingewikkelder dan het lijkt omdat we voor iedere rij in deze nieuwe dataset in drie verschillende rijen in de oude dataset moeten gaan zoeken. Maar zo een uitdaging ga ik graag aan en na mijn hoofd er een tijdje op te breken ben ik wel op een oplossing gekomen. Voor het algoritme gebruik ik onderstaande functie.

De functie krijgt een lijst van dictionaries mee als parameter. Die dictionaries komen overeen met de rijen uit de ingelezen dataset en met een for-lus wordt over deze rijen geïterreerd. Buiten de for-lus wordt een nieuwe dictionary aangemaakt waar de hervormde data in wordt opgeslagen.

Binnen de for-lus wordt opnieuw een lus opgezet om de verschillende jaren te doorlopen. Hierin wordt steeds een specifieke rij van de hervormde data aangemaakt of bijgewerkt. In de nieuwe dataset zal de combinatie van 'Area Code', 'Item Code' en het jaartal uniek zijn voor iedere rij. Deze wordt dan ook gebruikt als _key_ om het onderscheid te maken tussen het aanmaken van een nieuwe rij of een bestaande rij te bewerken.

Als de gemaakte _key_ nog niet terug te vinden is, wil dat zeggen dat die rij nog niet bestaat. Deze wordt dan aangemaakt en alle gegevens behalve die van 'Element' en de bijhorende waarde voor dat jaar. Achteraf wordt 'Element' gebruikt als kolom en wordt de bijhorende waarde voor dat jaar gebruikt als waarde voor die kolom. Dat gebeurd ook als de _key_ wel al gevonden was, op die manier wordt die rij aangepast zodat uiteindelijk alle data voor de verschillende mogelijkheden uit 'Element' in dezelfde rij zijn opgeslagen.

Enkel de kolommen 'Unit' en 'Element Code' uit de oude set gaan zo verloren, maar de waardes hiervan zijn 1 op 1 gelinkt aan de waardes in 'Element' waardoor die kolommen ook niet meer in de nieuwe dataset zouden passen.

In [36]:
def reform_dataset(df_dict):
    new_dict = {}

    for record in df_dict:
        for year in range(1961,2015):
            sep = '_'
            key = sep.join([str(record['Area Code']), str(record['Item Code']), str(year)])

            if key not in new_dict.keys():
                new_row = {
                    'Key': key,
                    'Area Code': record['Area Code'],
                    'Area': record['Area'],
                    'Item Code': record['Item Code'],
                    'Item': record['Item'],
                    'Year': year,
                    'YearF': record['Y'+str(year)+'F']
                       }
                new_dict[key] = new_row
            
            column = record['Element']
            new_dict[key][column] = record['Y'+str(year)]


    return pd.DataFrame.from_dict(new_dict, orient='index').reset_index().drop(columns=['Key', 'index'])

Met dit algoritme kan de data naar wens hervormd worden. Daarvoor worden de lijst van continenten en de PATH-variabele gebruikt om de hervorming voor alle bestanden door te voeren.

Voor ieder continent wordt dus het bijhorende csv-bestand in een pandas dataframe ingeladen. Daarop wordt dan de vorige functie uitgevoerd zodat de data hervormd worden. 

Dankzij deze stappen is het aantal kolommen gezakt van 115 naar 10, nog voor het echte 'cleanen' van start is gegaan. Let wel dat er nu uiteraard veel meer rijen zijn.

In [34]:
for continent in continents:
    df = pd.read_csv(PATH + continent + '.csv', engine='python')
    df_reformed = reform_dataset(df.to_dict('records'))
    df_reformed.to_csv(PATH + continent + '_reformed.csv', index=False, header=True)

De hervormde versie van Oceanië wordt opnieuw ingeladen om verder te testen.

In [74]:
# Temporary cell for testing with 1 file

df = pd.read_csv(PATH + 'Oceania_reformed.csv', engine='python')

## Data Cleaning

Nu de opmaak van de dataset verbeterd is, is het ook duidelijker welke data er allemaal beschikbaar is en het tijd is voor de volgende stap: _data cleaning_. De stappen in dit deel zijn altijd vergelijkbaar over verschillende projecten, maar de exacte beslissingen die gemaakt worden zijn afhankelijk van zowel de data als het doel van het project. Om dat het nu voornamelijk de bedoeling is om de verschillende stappen te tonen, zijn de keuzes minder van belang. Wel wil ik graag tonen wat de mogelijkheden zijn.

### Noisy data
Omdat het nu duidelijk is welke data beschikbaar is, is het ook het duidelijker welke data al dan niet waardevol is en dat brengt ons bij het eerste deel van _data cleaning_: _noisy data_.

_Noisy data_ kan op twee dingen wijzen. Het kan gaan om individuele datapunten, in dit geval spreken we van _outliers_ die bijvoorbeeld op een foute meting kunnen wijzen. Anderzijds kan ook een _feature_ als _noisy data_ worden gezien. Dat is de optie die hier geldt; niet alle kolommen in het dataframe hebben namelijk een meerwaarde om analyses te kunnen uitvoeren.

Zo zijn er de kolommen 'Area' en 'Area Code' waarvan de waardes dezelfde info geven, gelijkaardig is er 'Item ' en 'Item Code'. Voor leesbaarheid zou het logisch zijn om voor de namen te gaan, maar als de data dan gebruikt wordt voor wiskundige modellen (wat meestal het geval is) zijn er nog extra stappen nodig voordat dat model met die kolommen om kan. Daarom koos ik er in dit geval voor om de codes te houden in de dataset en de combinatie van naam en code in een apart dataframe op te slaan en weg te schrijven. Op die manier kunnen de namen in 'Area' en 'Item' toch nog worden teruggevonden.

Daarnaast zijn ook de kolommen 'Yield' en 'YearF' overbodig. 'Yield' is berekend uit 'Production' en 'Area harvested', wat wil weggen dat deze waarden kunnen worden berekend. In dat geval wordt die kolom niet bijgehouden. De andere kolom, 'YearF', geeft de accuraatheid van de gegevens voor die rij aan. Omdat we zelf geen betere schatting gaan kunnen maken dan de makers van deze dataset kunnen we ook deze dataset eigenlijk laten vallen.

__Noisy data__

- As a feature
    - Kijken naar meerwaarde van de kolommen in het verhaal
    - Code vs naam
    - Yield, YearF, Seed?
    

Een tweede aspect van _data cleaning_ is _missing values_. Dit is een probleem dat zich in veel projecten voordoet en waar helaas niet echt een goede aanpak voor is. _Missing values_ zijn terug te vinden als _null_ of NaN-waardes en de beste keuze om hiermee om te gaan is afhankelijk van de hoeveelheid data en de hoeveelheid _missing values_.

Als er veel data beschikbaar is kan het een optie zijn om per rij te gaan kijken hoeveel waardes ontbreken. De kolommen 

__Missing Values__

    - Te veel missing values
        - rij verwijderen
    - Invullen met vervangende waarde
        - waarde is afhankelijk van het probleem
        - VB kolom 'Seed': NaN wijst erop dat er geen seeds zijn voor dit item, dus vervangen door 0

In [17]:
df_areas = pd.DataFrame(columns=['Area', 'Area Code'])
df_items = pd.DataFrame(columns=['Item', 'Item Code'])

for continent in continents:
    df = pd.read_csv(PATH + continent + '_reformed.csv', engine='python')
    
    areas = df[['Area', 'Area Code']].drop_duplicates()
    df_areas = pd.concat([df_areas, areas])
    items = df[['Item', 'Item Code']].drop_duplicates()
    df_items = pd.concat([df_items, items]).drop_duplicates()
    
    df.drop(columns=['Yield', 'YearF', 'Item', 'Area'], inplace=True)
    
    df['Seed'].fillna(0, inplace=True)
    df.dropna(subset=['Area harvested', 'Production'], how='all', inplace=True)
    df.fillna(method='ffill', inplace=True)
    
    df.to_csv(PATH + continent + '_cleaned.csv', index=False, header=True)
    
df_areas.to_csv(PATH + 'areas.csv', index=False, header=True)
df_items.to_csv(PATH + 'items.csv', index=False, header=True)

(45, 2) (45, 2)
(168, 2) (168, 2)
(104, 2) (59, 2)
(175, 2) (164, 2)
(155, 2) (51, 2)
(182, 2) (172, 2)
(201, 2) (46, 2)
(182, 2) (145, 2)
(223, 2) (22, 2)
(182, 2) (133, 2)


## Feature scaling

Nu de data is opgeschoond, is de volgende stap vaak feature scaling. Het is van ontzettend belang voor bepaalde modellen. Meerbepaald van modellen die de afstanden tussen punten gaan berekenen in hun werking. Als niet alle variabelen in de data op dezelfde schaal zitten, kan dit die afstand namelijk gaan beïnvloeden. De bedoeling van _feature scaling_ is daarom het herschalen van de variabelen zodat ze allemaal op dezelfde schaal zitten.

### Waarom herschalen?

Als het gekozen model gebruik maakt van de afstanden tussen datapunten, heeft het herschalen van de variabelen een rechtstreekse invloed op de kwaliteit van het model. Neem als voorbeeld het aantal wielen van een voertuig en het gewicht van een voertuig in kilogram. Dit is een extreem voorbeeld, maar het is duidelijk dat deze variabelen niet op dezelfde schaal zitten. Het gewicht zal haast altijd veel hoger liggen en daarnaast zijn de waarden voor het gewicht veel breder verspreid. Bij de berekening van de afstanden heeft het gewicht dus veel meer invloed op het resultaat, waardoor een model er van uit gaat dat deze variabele van meer belang is. Door de variabelen te herschalen, zijn ze van gelijk belang voor het model.

### Wat doet het?

Er valt ontzettend veel te vertellen over _feature scaling_ en de verschillende technieken ervoor, om de lengte van dit artikel te bedrukken beperk ik me hier tot de uitleg van de twee belangrijkste technieken: normalisatie en standaardisatie. 

Bij normalisatie worden alle gegevens herschaalt naar waardes tussen twee getallen, meestal 0 en 1. Bij standaardisatie worden alle gegevens herschaalt zodat iedere variabele een gemiddelde waarde 0 heeft en standaardafwijking 1. Het is belangrijk om deze uit elkaar te kunnen houden, want vaak worden de twee termen door elkaar gebruikt.

De meest bekende normalisatie techniek is Min-Max normalisatie. Hierbij komen alle waardes tussen 0 en 1 te liggen door volgende formule die gebruik maakt van de minimale en maximale waarde van een variabele:

<img src="min-max-normalisation.jpg" alt="drawing" style="width:200px;" />

Voor standaardisatie is de Z-score de meest bekende. Voor iedere variabele wordt de gemiddelde waarde en de standaardafwijking gebruikt om te herschalen. Volgende formule resulteert dan zoals gezegd in een nieuwe reeks met gemiddelde 0 en standaardafwijing 1:

<img src="zscore.png" alt="drawing" style="width:200px;"/>

Welke de beste keuze van de twee is is volledig afhankelijk van het project en het gekozen model. Het is dus niet zo dat de een beter werkt dan de ander. De beste optie is wellicht om voor beide opties de resultaten te vergelijken en zo een keuze te maken.



## Verdere mogelijkheden

Ook nadat de data is opgeschoond en alle variabelen op eenzelfde schaal zitten, zijn er nog mogelijkheden om de data te verbeteren. Die mogelijkheden zijn zeer sterk afhankelijk van de data zelf, het project en het model. Omdat dit hier dus niet concreet genoeg is, worden andere nogelijkheden niet verder toegepast binnen deze dataset.

Volgende technieken kunnen interessant zijn voor een eigen project:

- __Clustering__: Rijen kunnen worden samengevoegd o.b.v. de vraag 
    - VB: Welke regio produceert het meeste producten? Groeperen op Area Code en gemiddelde van iedere regio
- __Under- en oversampling__: Bij classificatie. Soms zijn niet alle klassen even sterk vertegenwoordigd in de dataset.
    - VB: Onderscheid maken tussen katten en honden, wanneer de dataset 500 afbeeldingen van katten heeft, maar slechts 100 afbeeldingen van honden

Zoals in dit artikel dus overlopen is, komt er heel wat voorbereidend werk kijken bij een dataproject. Er kruipt in de praktijk naast het opstellen van deze stappen ook heel wat tijd in het leren kennen van de data en in het denkproces om tot de juiste stappen te komen. Vaak kruipt er zelfs meer tijd in dat voorbereidende werk dan in bijvoorbeeld het opstellen en trainen van het machine learning model, omdat goede data essentieel is voor zo een model.

